# Add embeddings to table

In this tutorial, we will take a set of previously computed embeddings and associate them with their corresponding bounding box in a Table representing
a COCO-style object-detection dataset.

This will create a Table with an additional column containing the embeddings for each bounding box.

To run this notebook, you must also have run:
* [1-fine-tune-on-crops.ipynb](https://github.com/3lc-ai/3lc-examples/blob/main/tutorials/bb-embeddings/1-fine-tune-on-crops.ipynb)
* [2-collect-embeddings.ipynb](https://github.com/3lc-ai/3lc-examples/blob/main/tutorials/bb-embeddings/2-collect-embeddings.ipynb)

## Imports

In [ ]:
from copy import deepcopy

import numpy as np
import tlc
from tqdm.auto import tqdm

## Project setup

In [ ]:
TRANSIENT_DATA_PATH = "../../../transient_data"
EMBEDDING_SAVE_PATH = TRANSIENT_DATA_PATH + "/bb_classifier_embeddings.npy"
LABELS_SAVE_PATH = TRANSIENT_DATA_PATH + "/bb_classifier_labels.npy"
BATCH_SIZE = 32
NUM_COMPONENTS = 3
EMBEDDING_COLUMN_NAME = "embedding"
PREDICTED_LABEL_COLUMN_NAME = "predicted_label"
OUTPUT_TABLE_NAME = "embeddings_added"
OUTPUT_TABLE_DESCRIPTION = "Added embeddings from fine-tuned bb-classifier"

## Load input Table

In [ ]:
input_table = tlc.Table.from_names(
    table_name="initial",
    dataset_name="COCO128",
    project_name="3LC Tutorials",
)

## Create new Table with embeddings

In [ ]:
# Open the embeddings and labels created in previous notebook
embeddings = np.load(EMBEDDING_SAVE_PATH)
labels = np.load(LABELS_SAVE_PATH)

# Create schema for embedding
embedding_schema = tlc.Schema(
    value=tlc.Float32Value(),
    size0=tlc.DimensionNumericValue(NUM_COMPONENTS, NUM_COMPONENTS),
)

# Create a schema for the new table
new_table_schema = deepcopy(input_table.rows_schema)
label_schema = deepcopy(new_table_schema.values["bbs"].values["bb_list"].values["label"])
label_schema.writable = False
new_table_schema.values["bbs"].values["bb_list"].add_sub_schema(EMBEDDING_COLUMN_NAME, embedding_schema)
new_table_schema.values["bbs"].values["bb_list"].add_sub_schema(PREDICTED_LABEL_COLUMN_NAME, label_schema)

In [ ]:
# Create a TableWriter for the new table
table_writer = tlc.TableWriter(
    project_name=input_table.project_name,
    dataset_name=input_table.dataset_name,
    table_name=OUTPUT_TABLE_NAME,
    description=OUTPUT_TABLE_DESCRIPTION,
    column_schemas=new_table_schema.values,
    input_tables=[input_table.url],
)

# Iterate over the rows of the original table and add the embeddings
i = 0
for row in tqdm(input_table):
    new_row = deepcopy(row)
    for bb in new_row["bbs"]["bb_list"]:
        bb[EMBEDDING_COLUMN_NAME] = embeddings[i].tolist()
        bb[PREDICTED_LABEL_COLUMN_NAME] = int(labels[i])
        i += 1

    table_writer.add_row(new_row)

# Create the new table
table_writer.finalize()